In [1]:
# Copyright 2023 Ivlev Alexander

import pyqpanda as pq
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

GlobalPhase = 0

def X_bit_mask(x_mask, q_list): #The bit mask of gates X.
    cir = pq.QCircuit()

    i = 0
    while (x_mask != 0):
        if(x_mask & 1):
            cir << pq.X(q_list[i])
        i = i + 1
        x_mask = x_mask >> 1
    
    return cir

def CNOT_lin(q0, c, q_list): #CNOT with nearest-neighbor qubit
    cir = pq.QCircuit()

    if(abs(q0 - c) == 1):
        cir << pq.CNOT(q_list[q0], q_list[c])
    else:
        i = 0
        while(q0 + i < c):
            cir << pq.CNOT(q_list[q0 + i], q_list[q0 + i + 1])
            i+=1

        i -= 2
        while(i > 0):
            cir << pq.CNOT(q_list[q0 + i], q_list[q0 + i + 1])
            i-=1

        while(q0 + i < c):
            cir << pq.CNOT(q_list[q0 + i], q_list[q0 + i + 1])
            i+=1

        i -= 2
        while(i > 0):
            cir << pq.CNOT(q_list[q0 + i], q_list[q0 + i + 1])
            i-=1

    return cir

def SWAP_(q1, q2, qlist): #Gate SWAP
    cir = pq.QCircuit()

    cir << pq.CNOT(qlist[q1], qlist[q2])
    cir << pq.CNOT(qlist[q2], qlist[q1])
    cir << pq.CNOT(qlist[q1], qlist[q2])

    return cir

def C2NOT(q0, q1, c, q_list): #Toffoli gate if P -> GP += 0, RZ -> GP += pi/8
    cir = pq.QCircuit()

    cir << pq.H(q_list[c])
    cir << CNOT_lin(q1, c, q_list)
    cir << pq.RZ(q_list[c], -np.pi/4)
    cir << CNOT_lin(q0, c, q_list)
    cir << pq.RZ(q_list[c], np.pi/4)
    cir << CNOT_lin(q1, c, q_list)
    cir << pq.RZ(q_list[c], -np.pi/4)
    cir << CNOT_lin(q0, c, q_list)
    cir << pq.RZ(q_list[q1], np.pi/4)
    cir << pq.RZ(q_list[c], np.pi/4)
    cir << CNOT_lin(q0, q1, q_list)
    cir << pq.RZ(q_list[q1], -np.pi/4)
    cir << pq.RZ(q_list[q0], np.pi/4)
    cir << CNOT_lin(q0, q1, q_list)
    cir << pq.H(q_list[c])

    global GlobalPhase 
    GlobalPhase += np.pi/8

    return cir

def CP_(phi, q, c, q_list): #controlled P (RZ -> GP += (-1)*phi/4)
    cir = pq.QCircuit()

    cir << pq.RZ(q_list[q], phi/2)
    cir << CNOT_lin(q, c, q_list)
    cir << pq.RZ(q_list[c], -phi/2)
    cir << CNOT_lin(q, c, q_list)
    cir << pq.RZ(q_list[c], phi/2)

    global GlobalPhase
    GlobalPhase += (-1)*phi/4

    return cir

def CnPhase(ang, list, c, q_list): #Multi controlled P (RZ -> GP += (-1)*ang/(2**(len(list) + 1)))
    cir = pq.QCircuit()

    if(len(list) == 1):
        cir << CP_(ang, list[0], c, q_list)

    for i in range(1, len(list)):
        cir << CP_(ang/(2**i), list[-i], c, q_list)
        cir << CnNOT(list[:-i], list[-i], q_list)

        cir << CP_(-ang/(2**i), list[-i], c, q_list)
        cir << CnNOT(list[:-i], list[-i], q_list)

    cir << CP_(ang/(2**(len(list) - 1)), list[0], c, q_list)

    return cir

def CnNOT(list, c, q_list): #Multi controlled X
    cir = pq.QCircuit()

    if(len(list) == 1):
        cir << CNOT_lin(list[0], c, q_list)
    elif(len(list) == 2):
        cir << C2NOT(list[0], list[1], c, q_list)
    else:
        cir << pq.H(q_list[c])
        cir << CnPhase(np.pi, list, c, q_list) #Multi controlled Z
        cir << pq.H(q_list[c])

    return cir

def norm(vec): #Normalization
    sum = 0
    for elem in vec:
        sum += abs(elem)**2
    sum = np.sqrt(sum)
    return [x / sum for x in vec]

def abs_ang(vec, q_list): #Angles for RY to construct a vector of modules
    step = len(vec)
    level_ang = []
    temp_vec = [abs(x) for x in norm(vec)]

    for i in range(len(q_list)):
        step //= 2
        level_ang.append([])
        new_vec = []
        for j in range(step):
            a = temp_vec[j]
            b = temp_vec[j+step]
            c = np.sqrt(a**2 + b**2)
            new_vec.append(c)
            if (c != 0):
                level_ang[i].append(2*np.arccos(a/c))
            else:
                level_ang[i].append(0)
        temp_vec = new_vec
    
    return(list(reversed(level_ang)))

def phase_ang(vec): #Angles for RZ to set the phase.
    level_ang = []

    for i in range(len(vec)):
        temp_ang = np.angle(vec[i])
        if (temp_ang < 0):
            temp_ang += (2*np.pi)
        level_ang.append(temp_ang)
    
    return(level_ang)

def run(vec, q_list): #The algorithm for preparing the initial state.
    global GlobalPhase
    cir = pq.QCircuit()
    N_ = len(q_list)
    
    #vec = norm(vec)

    #Preparation of angles
    level_angles = abs_ang(vec, q_list)
    phase = phase_ang(vec)

    angle = level_angles[0]
    if(angle[0] != 0):
        cir << pq.RY(q_list[0], angle[0])
    if(N_ == 1):
        if(phase[1] != 0):
            cir << pq.RZ(q_list[0], -phase[0] + phase[1])
        GlobalPhase += (phase[0] + phase[1])/2 #GP not equal to 0 for N = 1.

    temp = [0]

    for i in range(1, N_):
        angles = level_angles[i]
        last_0 = 2**i - 1 

        for j in range(2**(i)):
            if(j == 0 or angles[j] != 0 or i == N_-1):
                cir << X_bit_mask((2**(i) - j - 1) ^ (2**(i) - last_0 - 1), q_list)
                last_0 = j

            if(i == N_-1 and phase[j] != 0):
                cir << pq.RZ(q_list[i], (-1.)*phase[j])
            if(i == N_-1 and phase[j+2**(i)] != 0):
                cir << pq.X(q_list[i])
                cir << pq.RZ(q_list[i], phase[j+2**(i)])
                cir << pq.X(q_list[i])

            cir << pq.RY(q_list[i], angles[j]/2)

            cir << CnNOT(temp, i, q_list)

            cir << pq.RY(q_list[i], (-1)*angles[j]/2)
            if(i == N_-1 and phase[j] != 0):
                cir << pq.RZ(q_list[i], phase[j])

            cir << CnNOT(temp, i, q_list)

            if(i == N_-1 and phase[j+2**(i)] != 0):
                cir << pq.X(q_list[i])
                cir << pq.RZ(q_list[i], (-1)*phase[j+2**(i)])
                cir << pq.X(q_list[i])

        cir << X_bit_mask(2**(i) - last_0 - 1, q_list) 
        temp.append(i)

    #Removing the residual global phase
    GlobalPhase = GlobalPhase % (2*np.pi)
    while(GlobalPhase - np.pi/2 >= -0.00001):
        cir << pq.X(q_list[0])
        cir << pq.Z(q_list[0])
        cir << pq.Y(q_list[0])
        GlobalPhase -= np.pi/2

    return cir


#QCM = pq.QCloud()
# Initialization by passing in the current user's token
#QCM.init_qvm("____", True)
pq.init(pq.QMachineType.CPU)

q = pq.qAlloc_many(2)#QCM.
c = pq.cAlloc_many(2)#QCM.

prog = pq.QProg()
quantum_state_vector = np.array(norm([1, 2, 3, 4]))
prog << run(quantum_state_vector, q)
prog << pq.Measure(q[0], c[0])\
    << pq.Measure(q[1], c[1])\
    #<< pq.Measure(q[2], c[2])

#result = QCM.real_chip_measure(prog, 2000, pq.real_chip_type.origin_wuyuan_d5)
result = pq.run_with_configuration(prog, c, 2000)
print(result)
pq.finalize()

{'00': 81, '01': 266, '10': 557, '11': 1096}


In [2]:
plt.bar(*zip(*result.items()))
plt.show()